In [74]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Excelデータの読み込み
file_path = 'data【10月】生産性結果記入用紙_Ver.4.3.xlsx'  # Excelファイルのパスを指定
sheet_name = '実績昼'  # 読み込むシート名を指定

data = pd.read_excel(file_path, sheet_name=sheet_name)
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,作業ライン,項目,2024-10-01 00:00:00,2024-10-02 00:00:00,2024-10-03 00:00:00,2024-10-04 00:00:00,2024-10-05 00:00:00,2024-10-06 00:00:00,2024-10-07 00:00:00,2024-10-08 00:00:00,...,2024-10-23 00:00:00,2024-10-24 00:00:00,2024-10-25 00:00:00,2024-10-26 00:00:00,2024-10-27 00:00:00,2024-10-28 00:00:00,2024-10-29 00:00:00,2024-10-30 00:00:00,2024-10-31 00:00:00,合計
2,A-PaLS,件数,0,1756,2125,2433,1490,0,2455,2921,...,2941,2498,2941,1249,0,2893,2264,297,170,57837
3,NaN,作業時間(分),0,4302,4740,7002,4620,0,5544,7026,...,8190,5871,7326,4485,0,6240,5694,771,486,154956
4,NaN,目標件数,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# 行列を逆にする
data = data.T
# 1行目を列名にする
data.columns = data.iloc[1]
# 0,1行目を削除
data = data.drop(data.index[[0, 1]])
data.head()


Unnamed: 1,NaN,項目,件数,作業時間(分),目標件数,作業人数,リーダー人数,作業ライン,1本あたりの平均作業数,予測,買い合わせ平均,NaN,NaN,項目,生産性(分/件),1日作業量/L
Unnamed: 2,NaN,2024-10-01 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
Unnamed: 3,NaN,2024-10-02 00:00:00,1756,4302,NaN,7,5,4,439,NaN,NaN,NaN,NaN,2,2.449886,177.559275
Unnamed: 4,NaN,2024-10-03 00:00:00,2125,4740,NaN,6,6,4,531.25,NaN,NaN,NaN,NaN,3,2.230588,195.015823
Unnamed: 5,NaN,2024-10-04 00:00:00,2433,7002,NaN,12,6,6,405.5,NaN,NaN,NaN,NaN,4,2.877928,151.150386
Unnamed: 6,NaN,2024-10-05 00:00:00,1490,4620,NaN,13,2,5,298,NaN,NaN,NaN,NaN,5,3.100671,140.292208


In [76]:
# 欠損値を探す
data.isnull().sum()


Unnamed: 1
NaN            32
項目              0
件数              0
作業時間(分)         0
目標件数           32
作業人数            4
リーダー人数          4
作業ライン           5
1本あたりの平均作業数     5
予測             32
買い合わせ平均        32
NaN            32
NaN            32
項目              0
生産性(分/件)        0
1日作業量/L         0
dtype: int64

In [66]:
# 0,1,4,7,8,9列目を削除
data = data.drop(data.columns[[0, 1,4,9,10]], axis=1)
data.head()

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,作業ライン,1本あたりの平均作業数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,NaN,NaN,NaN,NaN,0,0
Unnamed: 3,1756,4302,7,5,4,439,2.449886,177.559275
Unnamed: 4,2125,4740,6,6,4,531.25,2.230588,195.015823
Unnamed: 5,2433,7002,12,6,6,405.5,2.877928,151.150386
Unnamed: 6,1490,4620,13,2,5,298,3.100671,140.292208


In [67]:
# NaNを全て0に変換
data = data.fillna(0)
data.head()

/var/folders/zn/w212lcms0dxb2bg4lyzl6pxm0000gn/T/ipykernel_62578/287054918.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(0)


Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,作業ライン,1本あたりの平均作業数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,0,0,0,0.00,0.000000,0.000000
Unnamed: 3,1756,4302,7,5,4,439.00,2.449886,177.559275
Unnamed: 4,2125,4740,6,6,4,531.25,2.230588,195.015823
Unnamed: 5,2433,7002,12,6,6,405.50,2.877928,151.150386
Unnamed: 6,1490,4620,13,2,5,298.00,3.100671,140.292208


In [68]:
# 0列目のデータを表示
data.iloc[:, 0]
data

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,作業ライン,1本あたりの平均作業数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,0,0,0,0.000000,0.000000,0.000000
Unnamed: 3,1756,4302,7,5,4,439.000000,2.449886,177.559275
Unnamed: 4,2125,4740,6,6,4,531.250000,2.230588,195.015823
Unnamed: 5,2433,7002,12,6,6,405.500000,2.877928,151.150386
Unnamed: 6,1490,4620,13,2,5,298.000000,3.100671,140.292208
Unnamed: 7,0,0,11,4,5,0.000000,0.000000,0.000000
Unnamed: 8,2455,5544,11,4,5,491.000000,2.258248,192.627165
Unnamed: 9,2921,7026,18,3,7,417.285714,2.405341,180.847566
Unnamed: 10,2138,7905,17,4,7,305.428571,3.697381,117.650854
Unnamed: 11,1921,7332,17,4,7,274.428571,3.816762,113.970949


In [73]:
# 33,34行目を削除
data = data.drop(data.index[[31, 32,33]])
data

IndexError: index 32 is out of bounds for axis 0 with size 32

In [70]:
# 正規化 -1から1の範囲に収める  
data2 = data.astype(float) # データ型をfloatに変換
data2 = (data - data.mean()) / data.std() # 正規化。各列の値から平均を引いて標準偏差で割る
data2

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,作業ライン,1本あたりの平均作業数,生産性(分/件),1日作業量/L
Unnamed: 2,-0.363171,-0.363298,-0.363720,-0.363444,-1.922984,-1.726215,-2.102112,-2.124905
Unnamed: 3,-0.186750,-0.201920,-0.257635,-0.050804,-0.374921,0.830986,0.125470,0.617816
Unnamed: 4,-0.149678,-0.185490,-0.272790,0.011724,-0.374921,1.368347,-0.073929,0.887464
Unnamed: 5,-0.118734,-0.100636,-0.181860,0.011724,0.399110,0.635846,0.514671,0.209884
Unnamed: 6,-0.213475,-0.189991,-0.166705,-0.238388,0.012094,0.009653,0.717202,0.042160
Unnamed: 7,-0.363171,-0.363298,-0.197015,-0.113332,0.012094,-1.726215,-2.102112,-2.124905
Unnamed: 8,-0.116523,-0.155330,-0.197015,-0.113332,0.012094,1.133889,-0.048779,0.850567
Unnamed: 9,-0.069706,-0.099736,-0.090930,-0.175860,0.786125,0.704499,0.084966,0.668610
Unnamed: 10,-0.148372,-0.066763,-0.106085,-0.113332,0.786125,0.052924,1.259766,-0.307576
Unnamed: 11,-0.170173,-0.088257,-0.106085,-0.113332,0.786125,-0.127652,1.368315,-0.364419


In [71]:
# 目的変数と説明変数の設定
# 'target_column'を目的変数として、他のすべてのカラムを説明変数とする
X = data2.drop(columns= '件数')
y = data2['件数']

# データセットをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレスト回帰モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

Mean Squared Error: 0.002968239054133404
R² Score: 0.7938334416231804


In [72]:
# 実際に件数を予測
X_new = X_test.iloc[0].values.reshape(1, -1)
y_new = model.predict(X_new)
# Predictedは予測値、Actualは実際の値、Errorは予測値と実際の値の差
print(f'Predicted: {y_new[0]}')
print(f'Actual: {y_test.iloc[0]}')
print(f'Error: {y_test.iloc[0] - y_new[0]}')
# 正規化する前の値に戻す
X_new = X_new * data.drop(columns='件数').std().values + data.drop(columns='件数').mean().values
y_new = y_new * data['件数'].std() + data['件数'].mean()
y_test = y_test * data['件数'].std() + data['件数'].mean()
print(f'Predicted: {y_new[0]}')
print(f'Actual: {y_test.iloc[0]}')
print(f'Error: {y_test.iloc[0] - y_new[0]}')

Predicted: -0.2456532337102196
Actual: -0.33333218933324366
Error: -0.08767895562302405
Predicted: 1169.7099999999982
Actual: 297.0
Error: -872.7099999999982


/Users/ryoma/BLP/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [60]:
# 特徴量重要度を表示
feature_importances = model.feature_importances_
features = X.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances})
df = df.sort_values(by='importance', ascending=False)
df

,features,importance
0,作業時間(分),0.431597
1,作業人数,0.209369
2,リーダー人数,0.169021
6,1日作業量/L,0.087944
4,1本あたりの平均作業数,0.053593
5,生産性(分/件),0.038048
3,作業ライン,0.010429
